<a href="https://colab.research.google.com/github/nxxk23/PROJECT/blob/main/FileLab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
hemo = pd.read_csv('/content/drive/MyDrive/Thesis Project Kidney/Hemodialysis.csv', encoding='latin1')

In [3]:
lab = pd.read_csv('/content/drive/MyDrive/Thesis Project Kidney/Lab.csv')

In [4]:
slope = pd.read_excel('/content/drive/MyDrive/Thesis Project Kidney/Assign/Assign 3: Re-CountSlope & Quartile/LabSlope.xlsx')

In [5]:
# Create a list from hemo['PatientUID']
patient_uid_list = hemo['PatientUID'].unique().tolist()
# Remove patient UIDs from lab['PatientUID'] that are not in the list
lab = lab[~lab['PatientUID'].isin(patient_uid_list)]
lab.shape

(17728574, 11)

In [6]:
lab0 = pd.DataFrame(lab) #1522

In [7]:
lab_location = lab['Location'].value_counts().reset_index()
lab_location.columns = ['Location', 'Location_Count']

lab_result = lab['ResultItemName'].value_counts().reset_index()
lab_result.columns = ['ResultItemName', 'ResultItemName_Count']

lab_request = lab['RequestItemName'].value_counts().reset_index()
lab_request.columns = ['RequestItemName', 'RequestItemName_Count']

columns_to_count = ['Location', 'ResultItemName', 'RequestItemName']

# Count unique values in the specified columns
lab_count = lab[columns_to_count].value_counts().reset_index()
lab_count.columns = ['Location', 'ResultItemName', 'RequestItemName', 'Count']

lab_df = pd.DataFrame(lab_count)

In [8]:
LabSummaryDF = pd.DataFrame(lab_df)

# **Slope**


> **ค่า x = ค่าจากวิสิท t ที่มาก่อน**




In [9]:
data = pd.DataFrame(slope)
data = data[['PatientUID','PatientVisitUID','Slopes']]
data = data.dropna()

In [10]:
df = pd.DataFrame(data)

# Sort the DataFrame by PatientUID and PatientVisitUID
df = df.sort_values(by=['PatientUID', 'PatientVisitUID'])

# Create a new column to store the previous slope value
df['PreviousSlope'] = df['Slopes'].shift(1)

# Update the Slopes column with the PreviousSlope values
df['Slopes'] = df['PreviousSlope']

# Drop the PreviousSlope column
df = df.drop(columns=['PreviousSlope'])
data = df.dropna()



---
> **TestCode**
>> **takes too much time to execute**

    # Assuming you have the 'data', 'lab', and 'LabSummaryDF' DataFrames

    lab1 = []  # Initialize the list to store lab values

    for pid, vid in zip(data['PatientUID'], data['PatientVisitUID']):
    # Filter labDF for the current patient ID and visit ID
    subsetLabDF = lab[(lab['PatientUID'] == pid) & (lab['PatientVisitUID'] == vid)]

    pVLabList = subsetLabDF['ResultItemName']  # Get the list of lab items

    for labValue in LabSummaryDF.loc[[0, 103], 'ResultItemName']:
        if labValue in pVLabList.values:
            # Get the corresponding ResultValue
            labResultValue = subsetLabDF.loc[subsetLabDF['ResultItemName'] == labValue, 'ResultValue'].values[0]
            lab1.append(labResultValue)
        else:
            lab1.append(None)  # Append None if labValue is not found

     Add the lab1 list as a new column in the 'data' DataFrame
    data['lab1'] = lab1
---




# **List of LabValue**


> **Replace Item with List then create new table to store the value**




In [11]:
# Define the mapping of LabSummaryDF indices to lab result DataFrames
mapping = {
    'lab1': [1, 1587],
    'lab2': [3, 129, 185, 312],
    'lab3': [0, 103],
    'lab4': [5, 115],
    'lab5': [2, 112],
    'lab6': [6, 118],
    'lab7': [4, 117],
    'lab8': [63, 194],
    'lab9': [31, 172],
    'lab10': [33, 175],
    'lab11': [21, 137],
    'lab12': [22, 3001],
    'lab13': [8, 100, 1736, 2577, 2856, 2980],
    'lab14': [13],
    'lab15': [15],
    'lab16': [159],
    'lab17': [198, 238, 584],
    'lab18': [154],
    'lab19': [30, 156, 177, 223, 277, 561],
    'lab20': [49, 153],
    'lab21': [23, 114],
    'lab22': [50, 130],
    'lab23': [34, 126],
    'lab24': [51, 131],
    'lab25': [220, 304, 539, 2909],
    'lab26': [191, 253],
    'lab27': [163],
    'lab28': [108, 397, 73, 305],
    'lab29': [176, 2796],
    'lab30': [218, 949],
    'lab31': [233, 1424],
    'lab32': [182, 473]
    # ... add more mappings as needed
}

# Create the mapping of lab DataFrames
lab_mapping = {}
for lab_name, indices in mapping.items():
    lab_mapping[lab_name] = LabSummaryDF[LabSummaryDF.index.isin(lab.index[indices])]

# Iterate through the lab_mapping dictionary and update 'ResultItemName' and 'RequestItemName' in lab DataFrame
for lab_name, lab_df in lab_mapping.items():
    result_item_names = lab_df['ResultItemName'].tolist()
    request_item_names = lab_df['RequestItemName'].tolist()
    lab.loc[lab['ResultItemName'].isin(result_item_names), 'ResultItemName'] = lab_name
    lab.loc[lab['RequestItemName'].isin(request_item_names), 'RequestItemName'] = lab_name


In [12]:
# Create a list of all the lab result names that were updated
updated_result_names = list(mapping.keys())

# Filter the lab DataFrame to keep only the rows with 'ResultItemName' in the updated names list
lab = lab[lab['ResultItemName'].isin(updated_result_names)]


In [13]:
lab['RequestItemName'].nunique()

30

In [14]:
lab['ResultItemName'].nunique()

32

# **WideTableLab**


In [15]:
# Initialize a dictionary to store the vital values
lab_dict = {}

# Iterate through unique ResultItemName values
for result_item in lab['ResultItemName'].unique():
    # Filter vital DataFrame for the current result_item
    subset_lab = lab[lab['ResultItemName'] == result_item]

    # Merge the subset_lab with data on PatientUID and PatientVisitUID
    merged_data = pd.merge(data, subset_lab[['PatientUID', 'PatientVisitUID', 'ResultValue', 'CWhen']], on=['PatientUID', 'PatientVisitUID'], how='left')

    # Rename the ResultValue column to match the result_item
    merged_data.rename(columns={'ResultValue': result_item}, inplace=True)

    # Update the lab_dict with the result_item as the key and merged_data as the value
    lab_dict[result_item] = merged_data

# Concatenate the dictionary values to get the final result
final_result = pd.concat(lab_dict.values(), axis=1)

# Drop duplicate columns (except for 'PatientUID', 'PatientVisitUID', and 'Slopes')
final_result = final_result.loc[:, ~final_result.columns.duplicated()]


## **Drop duplicate column**

In [16]:
# Sort data by 'CWhen' in descending order within each patient's visit and keep the latest visit row
final_result.sort_values(by=['PatientUID', 'PatientVisitUID', 'CWhen'], ascending=[True, True, False], inplace=True)
final_result.drop_duplicates(subset=['PatientUID', 'PatientVisitUID'], keep='first', inplace=True)

In [17]:
final_result

,PatientUID,PatientVisitUID,Slopes,lab11,CWhen,lab2,lab9,lab21,lab3,lab19,...,lab20,lab7,lab6,lab4,lab28,lab27,lab32,lab29,lab30,lab31
0,2299.0,4286612.0,0.002857,NaN,NaN,19.0,NaN,217,0.84,83,...,5.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2299.0,4546471.0,0.020833,NaN,NaN,13.0,NaN,211,0.90,86,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2299.0,4801938.0,-0.062921,NaN,NaN,16.1,NaN,219,0.96,86,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2299.0,5051781.0,-0.056522,NaN,NaN,NaN,NaN,206,0.91,95,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2299.0,5296557.0,0.047619,NaN,NaN,14.1,NaN,NaN,1.01,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
329880,3432056.0,10235244.0,-0.208313,4.4,2566-01-09 12:30:08.737,22.0,NaN,NaN,3.04,NaN,...,NaN,NaN,102,134,NaN,NaN,NaN,NaN,NaN,NaN
329881,3432056.0,10306813.0,0.168400,NaN,NaN,13.8,NaN,NaN,3.16,NaN,...,NaN,26.2,NaN,139,NaN,NaN,NaN,NaN,NaN,NaN
329885,3432056.0,10408954.0,1.129412,3.9,2566-02-26 07:25:17.043,20.8,NaN,NaN,3.07,NaN,...,NaN,25.5,NaN,140,NaN,NaN,NaN,NaN,NaN,NaN
329886,3432056.0,10573233.0,0.035000,4.1,2566-04-10 09:51:31.020,20.7,NaN,NaN,2.55,NaN,...,NaN,24.9,NaN,143,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
labDF = final_result[['PatientUID','PatientVisitUID','Slopes','lab1','lab2', 'lab3',
                      'lab4', 'lab5', 'lab6', 'lab7', 'lab8', 'lab9', 'lab10', 'lab11',
                      'lab12', 'lab13', 'lab14', 'lab15', 'lab16', 'lab17', 'lab18',
                      'lab19', 'lab20', 'lab21', 'lab22', 'lab23', 'lab24', 'lab25', 'lab26',
                      'lab27', 'lab28', 'lab29', 'lab30', 'lab31', 'lab32']]
# Drop rows with missing values in 'PatientUID', 'PatientVisitUID', and 'Slopes'
labDF = labDF.dropna(subset=['PatientUID', 'PatientVisitUID', 'Slopes'])

In [19]:
labDF

,PatientUID,PatientVisitUID,Slopes,lab1,lab2,lab3,lab4,lab5,lab6,lab7,...,lab23,lab24,lab25,lab26,lab27,lab28,lab29,lab30,lab31,lab32
0,2299.0,4286612.0,0.002857,69.20,19.0,0.84,NaN,NaN,NaN,NaN,...,144,75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2299.0,4546471.0,0.020833,63.60,13.0,0.90,NaN,NaN,NaN,NaN,...,143,70,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2299.0,4801938.0,-0.062921,58.40,16.1,0.96,NaN,NaN,NaN,NaN,...,146,73,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2299.0,5051781.0,-0.056522,62.40,NaN,0.91,NaN,NaN,NaN,NaN,...,133,86,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2299.0,5296557.0,0.047619,55.00,14.1,1.01,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
329879,3409733.0,10435209.0,-0.655833,35.90,25.6,2.61,133,4.6,98,26.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
329880,3432056.0,10235244.0,-0.208313,49.20,22.0,3.04,134,4.8,102,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
329881,3432056.0,10306813.0,0.168400,68.80,13.8,3.16,139,4.3,NaN,26.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
329885,3432056.0,10408954.0,1.129412,46.90,20.8,3.07,140,4.8,NaN,25.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## **Rename Columns**

In [20]:
# Create a dictionary for renaming columns
column_rename_dict = {
    'lab1': 'eGFR',
    'lab2': 'BUN',
    'lab3': 'Creatinine',
    'lab4': 'Sodium',
    'lab5': 'Potassium',
    'lab6': 'Chloride',
    'lab7': 'Bicarbonate',
    'lab8': 'Uric',
    'lab9': 'Calcium',
    'lab10': 'Phosphorus',
    'lab11': 'Albumin',
    'lab12': 'Hemoglobin',
    'lab13': 'Hematocrit',
    'lab14': 'WBC',
    'lab15': 'Platelet',
    'lab16': 'TIBC',
    'lab17': 'Tranferin SAT',
    'lab18': 'Ferritin',
    'lab19': 'Blood sugar',
    'lab20': 'HbA1c',
    'lab21': 'Cholesterol',
    'lab22': 'Triglyceride',
    'lab23': 'LDL Cholesterol',
    'lab24': 'HDL Cholesterol',
    'lab25': 'Parathyroid hormone',
    'lab26': 'Vitamin D',
    'lab27': 'UrineAlbumin/Cr ratio',
    'lab28': 'UrineProtein/Cr ratio',
    'lab29': 'UrineVolume (24hrs)',
    'lab30': 'UrineUrea (24hrs)',
    'lab31': 'UrineSodium (24hrs)',
    'lab32': 'UrineProtein (24hrs)'
}
# Use the rename method to rename the columns
labDF.rename(columns=column_rename_dict, inplace=True)
labDF

,PatientUID,PatientVisitUID,Slopes,eGFR,BUN,Creatinine,Sodium,Potassium,Chloride,Bicarbonate,...,LDL Cholesterol,HDL Cholesterol,Parathyroid hormone,Vitamin D,UrineAlbumin/Cr ratio,UrineProtein/Cr ratio,UrineVolume (24hrs),UrineUrea (24hrs),UrineSodium (24hrs),UrineProtein (24hrs)
0,2299.0,4286612.0,0.002857,69.20,19.0,0.84,NaN,NaN,NaN,NaN,...,144,75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2299.0,4546471.0,0.020833,63.60,13.0,0.90,NaN,NaN,NaN,NaN,...,143,70,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2299.0,4801938.0,-0.062921,58.40,16.1,0.96,NaN,NaN,NaN,NaN,...,146,73,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2299.0,5051781.0,-0.056522,62.40,NaN,0.91,NaN,NaN,NaN,NaN,...,133,86,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2299.0,5296557.0,0.047619,55.00,14.1,1.01,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
329879,3409733.0,10435209.0,-0.655833,35.90,25.6,2.61,133,4.6,98,26.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
329880,3432056.0,10235244.0,-0.208313,49.20,22.0,3.04,134,4.8,102,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
329881,3432056.0,10306813.0,0.168400,68.80,13.8,3.16,139,4.3,NaN,26.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
329885,3432056.0,10408954.0,1.129412,46.90,20.8,3.07,140,4.8,NaN,25.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


> **Check dup**

In [21]:
# Check for duplicate rows in data
duplicates_data = data[data.duplicated(['PatientUID', 'PatientVisitUID'], keep=False)]

# Check for duplicate rows in vitalDF
duplicates_vitalDF = labDF[labDF.duplicated(['PatientUID', 'PatientVisitUID', 'Slopes'], keep=False)]


In [22]:
duplicates_vitalDF

,PatientUID,PatientVisitUID,Slopes,eGFR,BUN,Creatinine,Sodium,Potassium,Chloride,Bicarbonate,...,LDL Cholesterol,HDL Cholesterol,Parathyroid hormone,Vitamin D,UrineAlbumin/Cr ratio,UrineProtein/Cr ratio,UrineVolume (24hrs),UrineUrea (24hrs),UrineSodium (24hrs),UrineProtein (24hrs)


In [23]:
labDF.to_excel('labDF.xlsx')